# Regression Discontinuity with Medicare Advantage Star Ratings

This notebook mirrors the **analysis** in the `Regression Discontinuity: Part II` slides, implemented in both **R** and **Python**, using the 2009 MA data (`ma_2009`).

## 1. Setup (R)

Load required R packages and the 2009 Medicare Advantage snippet data.

In [ ]:
%%R
if (!require("pacman")) install.packages("pacman")
pacman::p_load(tidyverse, ggplot2, lubridate, here, modelsummary,
               rddensity, rdrobust, MatchIt, cobalt, scales)

# Load 2009 Medicare Advantage snippet
ma_2009 <- readr::read_csv("../data/output/ma-snippets/ma-data-2009.csv") %>%
  mutate(mkt_share = avg_enrollment / avg_enrolled)


## 2. Setup (Python)

Load the same 2009 data in Python. If you have the `rdrobust` Python package installed, you can run the RD routines in Python as well.

In [ ]:
import pandas as pd
import numpy as np

# Optional: for Python RD implementations
# !pip install rdrobust

try:
    from rdrobust import rdrobust, rdplot, rddensity, rdplotdensity
    rd_available = True
except ImportError:
    rd_available = False
    print("Python rdrobust package not found. Install it with `pip install rdrobust` or use R code instead.")

# Load 2009 data
ma_2009 = pd.read_csv("../data/output/ma-snippets/ma-data-2009.csv")
ma_2009["mkt_share"] = ma_2009["avg_enrollment"] / ma_2009["avg_enrolled"]


## 3. Simple Relationship Between Star Ratings and Market Share

We start with a simple OLS specification regressing market share on star-rating dummies.

### 3.1 R: OLS with Star-Rating Dummies

In [ ]:
%%R
rating.ols <- lm(mkt_share ~ factor(Star_Rating), data = ma_2009)
summary(rating.ols)


### 3.2 Python: OLS with Star-Rating Dummies

In [ ]:
import statsmodels.formula.api as smf

rating_ols = smf.ols("mkt_share ~ C(Star_Rating)", data=ma_2009).fit()
print(rating_ols.summary())


## 4. RD Design Around the 2 vs 2.5-Star Threshold (Cutoff at 2.25)

Construct a sample around the 2 vs 2.5-star cutoff using the continuous raw score (`raw_rating`).

### 4.1 R: Constructing the RD Sample

In [ ]:
%%R
# Candidates: partc_score 2 or 2.5 with non-missing raw_rating and partc_score
ma_25star_candidates <- ma_2009 %>%
  filter(
    !is.na(raw_rating),
    !is.na(partc_score),
    Star_Rating %in% c(2.0, 2.5)
  )

n_candidates_total <- nrow(ma_25star_candidates)
n_candidates_by_score <- ma_25star_candidates %>% count(partc_score)

# Final sample: raw_rating in the range consistent with the star score
ma_25star <- ma_25star_candidates %>%
  filter(
    raw_rating >= 2,
    raw_rating <= 2.5,
    (raw_rating >= 2.25 & Star_Rating == 2.5) |
      (raw_rating <  2.25 & Star_Rating == 2)
  )

n_25star_total <- nrow(ma_25star)
n_25star_by_score <- ma_25star %>% count(partc_score)

cat("Obs with partc_score 2 or 2.5:", n_candidates_total, "\n")
print(n_candidates_by_score)
cat("Obs with raw_rating in the matching range:", n_25star_total, "\n")
print(n_25star_by_score)


### 4.2 Python: Constructing the RD Sample

In [ ]:
# Candidates: partc_score 2 or 2.5 with non-missing raw_rating and partc_score
candidates_mask = (
    ma_2009["raw_rating"].notna()
    & ma_2009["partc_score"].notna()
    & ma_2009["Star_Rating"].isin([2.0, 2.5])
)
ma_25star_candidates = ma_2009.loc[candidates_mask].copy()

n_candidates_total = len(ma_25star_candidates)
n_candidates_by_score = ma_25star_candidates["partc_score"].value_counts()

# Final sample: raw_rating in the range consistent with the star score
match_mask = (
    ma_25star_candidates["raw_rating"].between(2, 2.5)
    & (
        ((ma_25star_candidates["Star_Rating"] == 2.5) &
         (ma_25star_candidates["raw_rating"] >= 2.25))
        |
        ((ma_25star_candidates["Star_Rating"] == 2.0) &
         (ma_25star_candidates["raw_rating"] < 2.25))
    )
)
ma_25star_py = ma_25star_candidates.loc[match_mask].copy()

n_25star_total_py = len(ma_25star_py)
n_25star_by_score_py = ma_25star_py["partc_score"].value_counts()

print("Obs with partc_score 2 or 2.5:", n_candidates_total)
print(n_candidates_by_score)
print("Obs with raw_rating in the matching range:", n_25star_total_py)
print(n_25star_by_score_py)


## 5. Binned Scatterplot (RD Plot)

Visualize the jump in market share around the 2.25 cutoff.

### 5.1 R: RD Plot Using `rdplot`

In [ ]:
%%R
rd.result <- rdplot(ma_25star$mkt_share, ma_25star$raw_rating,
                    c = 2.25, p = 2,
                    title = "RD Plot with Binned Average",
                    x.label = "Running Variable",
                    y.label = "Outcome",
                    hide = TRUE)
bin.avg <- as_tibble(rd.result$vars_bins)
plot.bin <- bin.avg %>% ggplot(aes(x = rdplot_mean_x, y = rdplot_mean_y)) +
  geom_point() + theme_bw() +
  geom_vline(aes(xintercept = 2.25), linetype = 'dashed') +
  scale_x_continuous(
    breaks = c(2, 2.5),
    label  = c("Untreated", "Treated")
  ) +
  xlab("Running Variable") + ylab("Outcome")
plot.bin


### 5.2 Python: RD Plot (if `rdrobust` is available)

In [ ]:
import matplotlib.pyplot as plt

if rd_available:
    rd_result = rdplot(
        y=ma_25star_py["mkt_share"].values,
        x=ma_25star_py["raw_rating"].values,
        c=2.25,
        p=2,
        title="RD Plot with Binned Average",
        x_label="Running Variable",
        y_label="Outcome",
        hide=True
    )

    bin_avg = rd_result.vars_bins

    fig, ax = plt.subplots()
    ax.scatter(bin_avg["rdplot_mean_x"], bin_avg["rdplot_mean_y"])
    ax.axvline(x=2.25, linestyle="--")
    ax.set_xticks([2.0, 2.5])
    ax.set_xticklabels(["Untreated", "Treated"])
    ax.set_xlabel("Running Variable")
    ax.set_ylabel("Outcome")
    ax.set_title("RD Plot with Binned Average")
    plt.tight_layout()
    plt.show()
else:
    print("rdrobust not available in Python; use the R code above for the RD plot.")


## 6. RD from OLS

Estimate local linear RD models with different bandwidths and slope restrictions.

### 6.1 R: Local Linear RD Regressions

In [ ]:
%%R
ma.rd1 <- ma_25star %>%
  mutate(score       = raw_rating - 2.25,
         treat       = (score >= 0),
         window1     = (score >= -0.175 & score <= 0.175),
         window2     = (score >= -0.125 & score <= 0.125),
         score_treat = score * treat)

star25.1 <- lm(mkt_share ~ score + treat, data = ma.rd1)
star25.2 <- lm(mkt_share ~ score + treat, data = (ma.rd1 %>% filter(window1 == TRUE)))
star25.3 <- lm(mkt_share ~ score + treat + score_treat, data = (ma.rd1 %>% filter(window1 == TRUE)))
star25.4 <- lm(mkt_share ~ score + treat + score_treat, data = (ma.rd1 %>% filter(window2 == TRUE)))

est1 <- as.numeric(star25.1$coef[3])
est2 <- as.numeric(star25.2$coef[3])
est3 <- as.numeric(star25.3$coef[3])
est4 <- as.numeric(star25.4$coef[3])

rows <- tribble(~term, ~m1, ~m2, ~m3, ~m4,
                'Bandwidth', "0.25", "0.175", "0.175", "0.125")
attr(rows, 'position') <- 7

modelsummary(list(star25.1, star25.2, star25.3, star25.4),
             keep     = c("score", "treatTRUE", "score_treat"),
             coef_map = c("score"       = "Raw Score",
                          "treatTRUE"   = "Treatment",
                          "score_treat" = "Score x Treat"),
             gof_map  = c("nobs", "r.squared"),
             add_rows = rows)


### 6.2 Python: Local Linear RD Regressions

In [ ]:
import statsmodels.formula.api as smf

ma_rd1 = ma_25star_py.copy()
ma_rd1["score"] = ma_rd1["raw_rating"] - 2.25
ma_rd1["treat"] = (ma_rd1["score"] >= 0).astype(int)
ma_rd1["window1"] = ma_rd1["score"].between(-0.175, 0.175)
ma_rd1["window2"] = ma_rd1["score"].between(-0.125, 0.125)
ma_rd1["score_treat"] = ma_rd1["score"] * ma_rd1["treat"]

star25_1 = smf.ols("mkt_share ~ score + treat", data=ma_rd1).fit()
star25_2 = smf.ols("mkt_share ~ score + treat", data=ma_rd1[ma_rd1["window1"]]).fit()
star25_3 = smf.ols("mkt_share ~ score + treat + score_treat", data=ma_rd1[ma_rd1["window1"]]).fit()
star25_4 = smf.ols("mkt_share ~ score + treat + score_treat", data=ma_rd1[ma_rd1["window2"]]).fit()

est1_py = float(star25_1.params["treat"])
est2_py = float(star25_2.params["treat"])
est3_py = float(star25_3.params["treat"])
est4_py = float(star25_4.params["treat"])

print(star25_1.summary().tables[1])
print(star25_2.summary().tables[1])
print(star25_3.summary().tables[1])
print(star25_4.summary().tables[1])


## 7. Built-in RD Packages

Use `rdrobust` to estimate the RD with a fixed bandwidth and with data-driven bandwidth selection.

### 7.1 R: `rdrobust` with Fixed and Optimal Bandwidths

In [ ]:
%%R
est1 <- rdrobust(y = ma.rd1$mkt_share, x = ma.rd1$score, c = 0,
                 h = 0.125, p = 1, kernel = "uniform", vce = "hc0")

estopt <- rdrobust(y = ma.rd1$mkt_share, x = ma.rd1$score, c = 0,
                   p = 1, kernel = "uniform", vce = "hc0")

summary(est1)
summary(estopt)


### 7.2 Python: `rdrobust` (If Available)

In [ ]:
if rd_available:
    est1_py = rdrobust(
        y=ma_rd1["mkt_share"].values,
        x=ma_rd1["score"].values,
        c=0,
        h=0.125,
        p=1,
        kernel="uniform",
        vce="hc0",
        masspoints="off",
    )

    estopt_py = rdrobust(
        y=ma_rd1["mkt_share"].values,
        x=ma_rd1["score"].values,
        c=0,
        p=1,
        kernel="uniform",
        vce="hc0",
        masspoints="off",
    )
    print("Fixed bandwidth estimate:")
    print(est1_py)
    print("\nOptimal bandwidth estimate:")
    print(estopt_py)
else:
    print("rdrobust not available in Python; use the R code above for RD estimation.")


## 8. Manipulation and Balance Checks

Check for manipulation of the running variable and covariate balance near the cutoff.

### 8.1 R: Density Test for Manipulation (McCrary)

In [ ]:
%%R
dens1 <- rddensity(ma.rd1$score, c = 0)
densplot <- rdplotdensity(dens1, ma.rd1$score)
summary(dens1)


### 8.2 Python: Density Test (If Available)

In [ ]:
if rd_available:
    dens1_py = rddensity(x=ma_rd1["score"].values, c=0)
    densplot_py = rdplotdensity(dens1_py, x=ma_rd1["score"].values)
else:
    print("rddensity/rdplotdensity not available in Python; use the R code above.")


### 8.3 Covariate Balance

Use matching in R and a Mahalanobis NN matching analogue in Python, then inspect balance.

#### 8.3.1 R: Matching with `MatchIt`

In [ ]:
%%R
match.dat <- matchit(treat ~ premium_partc + ma_rate,
                     data = ma.rd1 %>%
                       filter(window2 == TRUE,
                              !is.na(treat),
                              !is.na(premium_partc),
                              !is.na(ma_rate)),
                     method   = NULL,
                     distance = "mahalanobis")


#### 8.3.2 Python: Mahalanobis NN Matching and Love-Plot Analogue

In [ ]:
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors

# Subset data as in the R code
sub = (
    ma_rd1
    .loc[
        (ma_rd1["window2"]) &
        ma_rd1["treat"].notna() &
        ma_rd1["premium_partc"].notna() &
        ma_rd1["ma_rate"].notna()
    ]
    .copy()
)

covariates = ["premium_partc", "ma_rate"]
X = sub[covariates].to_numpy()
treat_vec = sub["treat"].astype(int).to_numpy()

# Mahalanobis NN matching (1:1)
V = np.cov(X, rowvar=False)
nn = NearestNeighbors(
    n_neighbors=1,
    metric="mahalanobis",
    metric_params={"V": V}
)
nn.fit(X[treat_vec == 0])

dist, idx = nn.kneighbors(X[treat_vec == 1])
control_matches = sub.loc[treat_vec == 0].iloc[idx[:, 0]].copy()
treated_matched = sub.loc[treat_vec == 1].copy()

matched = pd.concat(
    [treated_matched.reset_index(drop=True),
     control_matches.reset_index(drop=True)],
    axis=0
).reset_index(drop=True)
matched["treat"] = np.r_[np.ones(len(treated_matched)), np.zeros(len(control_matches))]

def smd(df, var, treat_col="treat"):
    g1 = df[df[treat_col] == 1][var]
    g0 = df[df[treat_col] == 0][var]
    m1, m0 = g1.mean(), g0.mean()
    s = np.sqrt(0.5 * (g1.var(ddof=1) + g0.var(ddof=1)))
    return (m1 - m0) / s

smd_before = [smd(sub, v) for v in covariates]
smd_after  = [smd(matched, v) for v in covariates]

fig, ax = plt.subplots()
y_pos = np.arange(len(covariates))
ax.scatter(np.abs(smd_before), y_pos, marker="o", label="Unmatched")
ax.scatter(np.abs(smd_after),  y_pos, marker="s", label="Matched")
ax.set_yticks(y_pos)
ax.set_yticklabels(covariates)
ax.set_xlabel("Absolute standardized mean difference")
ax.axvline(0.1, linestyle="--")
ax.legend()
ax.invert_yaxis()
plt.tight_layout()
plt.show()


#### 8.3.3 R: Love Plot Output

In [ ]:
%%R
love.plot(match.dat, abs = TRUE)


## 9. Effects Across Rating Thresholds

Estimate RD effects at several thresholds in 2009: 2 vs 2.5, 2.5 vs 3, 3 vs 3.5, and 3.5 vs 4.

### 9.1 2.5 versus 2.0 Stars (Cutoff at 2.25)

#### R

In [ ]:
%%R
ma.rd225 <- ma_2009 %>%
  filter(Star_Rating == 2 | Star_Rating == 2.5) %>%
  mutate(score       = raw_rating - 2.25,
         treat       = (score >= 0),
         window1     = (score >= -0.175 & score <= 0.175),
         window2     = (score >= -0.125 & score <= 0.125),
         score_treat = score * treat)

est225 <- rdrobust(y = ma.rd225$mkt_share, x = ma.rd225$score, c = 0,
                   h = 0.125, p = 1, kernel = "uniform", vce = "hc0",
                   masspoints = "off")
summary(est225)


#### Python

In [ ]:
if rd_available:
    ma_rd225 = (
        ma_2009
        .loc[ma_2009["Star_Rating"].isin([2, 2.5])]
        .assign(
            score=lambda d: d["raw_rating"] - 2.25,
            treat=lambda d: (d["score"] >= 0).astype(int),
            window1=lambda d: d["score"].between(-0.175, 0.175),
            window2=lambda d: d["score"].between(-0.125, 0.125),
            score_treat=lambda d: d["score"] * d["treat"],
        )
    )

    est225_py = rdrobust(
        y=ma_rd225["mkt_share"].to_numpy(),
        x=ma_rd225["score"].to_numpy(),
        c=0,
        h=0.125,
        p=1,
        kernel="uniform",
        vce="hc0",
        masspoints="off"
    )
    print(est225_py)
else:
    print("rdrobust not available in Python; see R block for estimation.")


### 9.2 3.0 versus 2.5 Stars (Cutoff at 2.75)

#### R

In [ ]:
%%R
ma.rd275 <- ma_2009 %>%
  filter(Star_Rating == 2.5 | Star_Rating == 3) %>%
  mutate(score       = raw_rating - 2.75,
         treat       = (score >= 0),
         window1     = (score >= -0.175 & score <= 0.175),
         window2     = (score >= -0.125 & score <= 0.125),
         score_treat = score * treat)

est275 <- rdrobust(y = ma.rd275$mkt_share, x = ma.rd275$score, c = 0,
                   h = 0.125, p = 1, kernel = "uniform", vce = "hc0",
                   masspoints = "off")
summary(est275)


#### Python

In [ ]:
if rd_available:
    ma_rd275 = (
        ma_2009
        .loc[ma_2009["Star_Rating"].isin([2.5, 3])]
        .assign(
            score=lambda d: d["raw_rating"] - 2.75,
            treat=lambda d: (d["score"] >= 0).astype(int),
            window1=lambda d: d["score"].between(-0.175, 0.175),
            window2=lambda d: d["score"].between(-0.125, 0.125),
            score_treat=lambda d: d["score"] * d["treat"],
        )
    )

    est275_py = rdrobust(
        y=ma_rd275["mkt_share"].to_numpy(),
        x=ma_rd275["score"].to_numpy(),
        c=0,
        h=0.125,
        p=1,
        kernel="uniform",
        vce="hc0",
        masspoints="off"
    )
    print(est275_py)
else:
    print("rdrobust not available in Python; see R block for estimation.")


### 9.3 3.5 versus 3.0 Stars (Cutoff at 3.25)

#### R

In [ ]:
%%R
ma.rd325 <- ma_2009 %>%
  filter(Star_Rating == 3 | Star_Rating == 3.5) %>%
  mutate(score       = raw_rating - 3.25,
         treat       = (score >= 0),
         window1     = (score >= -0.175 & score <= 0.175),
         window2     = (score >= -0.125 & score <= 0.125),
         score_treat = score * treat)

est325 <- rdrobust(y = ma.rd325$mkt_share, x = ma.rd325$score, c = 0,
                   h = 0.125, p = 1, kernel = "uniform", vce = "hc0",
                   masspoints = "off")
summary(est325)


#### Python

In [ ]:
if rd_available:
    ma_rd325 = (
        ma_2009
        .loc[ma_2009["Star_Rating"].isin([3, 3.5])]
        .assign(
            score=lambda d: d["raw_rating"] - 3.25,
            treat=lambda d: (d["score"] >= 0).astype(int),
            window1=lambda d: d["score"].between(-0.175, 0.175),
            window2=lambda d: d["score"].between(-0.125, 0.125),
            score_treat=lambda d: d["score"] * d["treat"],
        )
    )

    est325_py = rdrobust(
        y=ma_rd325["mkt_share"].to_numpy(),
        x=ma_rd325["score"].to_numpy(),
        c=0,
        h=0.125,
        p=1,
        kernel="uniform",
        vce="hc0",
        masspoints="off"
    )
    print(est325_py)
else:
    print("rdrobust not available in Python; see R block for estimation.")


### 9.4 4.0 versus 3.5 Stars (Cutoff at 3.75)

#### R

In [ ]:
%%R
ma.rd375 <- ma_2009 %>%
  filter(Star_Rating == 3.5 | Star_Rating == 4) %>%
  mutate(score       = raw_rating - 3.75,
         treat       = (score >= 0),
         window1     = (score >= -0.175 & score <= 0.175),
         window2     = (score >= -0.125 & score <= 0.125),
         score_treat = score * treat)

est375 <- rdrobust(y = ma.rd375$mkt_share, x = ma.rd375$score, c = 0,
                   h = 0.125, p = 1, kernel = "uniform", vce = "hc0",
                   masspoints = "off")
summary(est375)


#### Python

In [ ]:
if rd_available:
    ma_rd375 = (
        ma_2009
        .loc[ma_2009["Star_Rating"].isin([3.5, 4])]
        .assign(
            score=lambda d: d["raw_rating"] - 3.75,
            treat=lambda d: (d["score"] >= 0).astype(int),
            window1=lambda d: d["score"].between(-0.175, 0.175),
            window2=lambda d: d["score"].between(-0.125, 0.125),
            score_treat=lambda d: d["score"] * d["treat"],
        )
    )

    est375_py = rdrobust(
        y=ma_rd375["mkt_share"].to_numpy(),
        x=ma_rd375["score"].to_numpy(),
        c=0,
        h=0.125,
        p=1,
        kernel="uniform",
        vce="hc0",
        masspoints="off"
    )
    print(est375_py)
else:
    print("rdrobust not available in Python; see R block for estimation.")


## 10. Wrap-Up

This notebook provides a working environment (in both R and Python) to:

- Construct RD samples around MA star-rating thresholds
- Visualize the discontinuity with binned scatterplots
- Estimate local linear RD models with OLS and `rdrobust`
- Check for manipulation of the running variable and covariate balance

All analyses here are based on the 2009 MA data (`ma_2009`).